In [2]:
import requests
import pandas as pd
import time
import os
import sys

# Add the 'scripts' folder to the Python path
sys.path.append(os.path.abspath(os.path.join('../scripts')))
from discord_notifier import send_discord_message

# Ensure the directory exists
def ensure_directory_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

# Load the existing summoner_ids from the entries.csv in the mass_fetch_8K_accounts folder
existing_entries_csv = '../data/mass_fetch_8K_accounts/entries.csv'
if os.path.exists(existing_entries_csv):
    existing_entries_df = pd.read_csv(existing_entries_csv)
    existing_summoner_ids = set(existing_entries_df['summoner_id'].unique())  # Set for fast lookup
else:
    existing_summoner_ids = set()

# Function to insert summoner data into a CSV file
def insert_summoner_data_bulk(summoner_data_list, csv_file='../data/raw/entries.csv'):
    # Ensure the directory exists
    ensure_directory_exists(os.path.dirname(csv_file))
    
    # Check if the file exists
    if os.path.exists(csv_file):
        df_summoner = pd.read_csv(csv_file)
    else:
        # If it doesn't exist, create an empty DataFrame with appropriate columns
        df_summoner = pd.DataFrame(columns=[
            'league_id', 'queue_type', 'tier', 'rank', 'summoner_id', 'league_points', 'wins', 'losses', 
            'veteran', 'inactive', 'fresh_blood', 'hot_streak'
        ])

    # Convert the summoner_data_list into a DataFrame
    new_data = pd.DataFrame(summoner_data_list, columns=df_summoner.columns)

    # Append new data and remove duplicates based on 'summoner_id'
    df_summoner = pd.concat([df_summoner, new_data]).drop_duplicates(subset=['summoner_id'])

    # Save the updated DataFrame back to CSV
    df_summoner.to_csv(csv_file, index=False)
    print(f"Inserted {len(new_data)} summoner entries into {csv_file}.")

# Function to pull summoner data from the Riot API and insert it into the CSV in bulk
def pull_summoner_data():
    api_key = "RGAPI-a6eedfcb-600e-4f76-b541-0a25e232aa36"
    base_url = "https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/GOLD/"
    divisions = ["II", "III", "IV"]

    for division in divisions:
        page = 30
        print(f"Getting values from page: {page} and division: {division}")
        player_count = 0  # Track the number of players pulled per division
        summoner_data_list = []  # List to store multiple entries for bulk insert
        while player_count < 500:
            
            url = f"{base_url}{division}?page={page}&api_key={api_key}"
            print(f"Fetching values from URL: {url}")
            response = requests.get(url)
            if response.status_code != 200:
                print("Error with the request:", response.status_code)
                break
            
            summoner_entries = response.json()
            print(f"Summoners length is: {len(summoner_entries)}")
            if not summoner_entries:
                break
            
            for entry in summoner_entries:
                if player_count >= 500:
                    break
                
                summoner_id = entry['summonerId']
                if summoner_id not in existing_summoner_ids:  # Only add new summoners
                    summoner_data = (
                        entry['leagueId'], entry['queueType'], entry['tier'], 
                        entry['rank'], entry['summonerId'], entry['leaguePoints'], 
                        entry['wins'], entry['losses'], entry['veteran'], 
                        entry['inactive'], entry['freshBlood'], entry['hotStreak']
                    )
                    summoner_data_list.append(summoner_data)
                    player_count += 1

            if summoner_data_list:
                insert_summoner_data_bulk(summoner_data_list)
                summoner_data_list = []  # Clear the list after inserting
            
            page += 1
            time.sleep(1.3)  # To respect the rate limit

if __name__ == "__main__":
    try:
        pull_summoner_data()
    except Exception as e:
        print(f"An error occurred: {e}")
    finally:
        send_discord_message()


Getting values from page: 30 and division: II
Fetching values from URL: https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/GOLD/II?page=30&api_key=RGAPI-a6eedfcb-600e-4f76-b541-0a25e232aa36
Summoners length is: 205
Inserted 202 summoner entries into ../data/raw/entries.csv.
Fetching values from URL: https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/GOLD/II?page=31&api_key=RGAPI-a6eedfcb-600e-4f76-b541-0a25e232aa36
Summoners length is: 205
Inserted 205 summoner entries into ../data/raw/entries.csv.
Fetching values from URL: https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/GOLD/II?page=32&api_key=RGAPI-a6eedfcb-600e-4f76-b541-0a25e232aa36
Summoners length is: 205
Inserted 93 summoner entries into ../data/raw/entries.csv.
Getting values from page: 30 and division: III
Fetching values from URL: https://na1.api.riotgames.com/lol/league/v4/entries/RANKED_SOLO_5x5/GOLD/III?page=30&api_key=RGAPI-a6eedfcb-600e-4f76-b541-0a25e232aa36
Summoner